# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - ORM (Object-Relational Mapping)

**Subtopic** - Understanding and Implementing Basic ORM with SQLAlchemy

**Use Case** - Define a SQLAlchemy column with a default value that is computed using a server-side function, ensuring consistency across database inserts.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt

```


# Prompt

**Problem Overview**
You are building a dynamic model generator that takes metadata about table columns and outputs a SQLAlchemy ORM model using `declarative_base()`. Each column in the table may include information about whether it should have a server-side default value. The default can apply to any column type, such as Integer, String, DateTime, Boolean, or UUID. Your task is to parse this metadata and generate a valid SQLAlchemy model with appropriate `Column` declarations using server-side defaults where applicable.


**Input Format**
A list of dictionaries. Each dictionary represents a column and contains the following keys:

* `name`: string, column name
* `type`: string, one of `"Integer"`, `"String"`, `"DateTime"`, `"Boolean"`, `"UUID"`, `"Float"`
* `primary_key`: boolean, whether this column is the primary key
* `nullable`: boolean, whether this column is nullable
* `server_default`: boolean, whether this column has a server-side default
* `default_function`: string or literal value

  * If string: treated as a SQL function or SQL expression (e.g. `"now"`, `"uuid_generate_v4"`, `"CURRENT_TIMESTAMP"`, `"'draft'"`)
  * If not string: treated as a raw literal (e.g. `0`, `True`, `'PENDING'`)


**Output Format**
A string representing a valid SQLAlchemy ORM model class with appropriate column definitions, including `server_default` attributes when specified. The model should use declarative syntax and generate only the necessary imports.


**Examples**

*Example 1*
**Input**

```python
[
    {
        "name": "id",
        "type": "UUID",
        "primary_key": True,
        "nullable": False,
        "server_default": True,
        "default_function": "uuid_generate_v4"
    },
    {
        "name": "status",
        "type": "String",
        "primary_key": False,
        "nullable": False,
        "server_default": True,
        "default_function": "'active'"
    },
    {
        "name": "created_at",
        "type": "DateTime",
        "primary_key": False,
        "nullable": False,
        "server_default": True,
        "default_function": "now"
    }
]
```

**Output**

```python
from sqlalchemy import Column, String, DateTime, text
from sqlalchemy.dialects.postgresql import UUID
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql import func

Base = declarative_base()

class GeneratedModel(Base):
    __tablename__ = 'generated_model'

    id = Column(UUID, primary_key=True, nullable=False, server_default=func.uuid_generate_v4())
    status = Column(String, nullable=False, server_default=text("'active'"))
    created_at = Column(DateTime, nullable=False, server_default=func.now())
```

**Explanation**

* The `id` field uses the SQL function `uuid_generate_v4()` via `func`.
* The `status` field uses a literal SQL string `'active'`.
* The `created_at` field uses `func.now()` for a server-side timestamp.

# Requirements


**Explicit Requirements**

* Use `server_default=func.<function>()` when the default is a known SQL function.
* Use `server_default=text('<literal>')` when the default is a literal SQL expression.
* The output model must include necessary imports based on input fields.
* The model class must be named `GeneratedModel`.
* The table name must be `'generated_model'`.


**Implicit Requirements**

* If `type` is `"String"`, assume a default length of 255 characters.
* Use only the necessary imports based on the fields present.
* Treat boolean literal defaults as `'TRUE'` or `'FALSE'` using SQL-compatible strings.
* Skip columns that contain unsupported types or incomplete metadata.


**Edge Cases**

* All columns have `server_default = False` -> No server defaults should appear in the output.
* Column has `server_default = True` but missing `default_function` -> Skip or raise a formatting error.
* Input list is empty -> Generate a class with just `__tablename__`.
* Unsupported or invalid types -> Skip or raise an error for that column.
* Literal default values like `0`, `True`, `False`, `'yes'` must be handled using SQL's `text()` wrapper.


**Function Signatures**

```python
def generate_sqlalchemy_model(columns: list[dict]) -> str:
    pass
```


**Constraints**

* Maximum of 100 columns in the input.
* Supported types: `"Integer"`, `"String"`, `"DateTime"`, `"Boolean"`, `"UUID"`, `"Float"`
* Supported SQL default functions include: `"now"`, `"CURRENT_TIMESTAMP"`, `"uuid_generate_v4"`, `"gen_random_uuid"`
* Output must follow PEP8 and PEP256 Python formatting standards.
* Model name must be `GeneratedModel` and table name must be `'generated_model'`.

In [ ]:
# code

def generate_sqlalchemy_model(columns: list[dict]) -> str:
    """
    Generates a SQLAlchemy model class using declarative base syntax
    from the given list of column definitions.

    Args:
        columns: List of dictionaries where each dictionary represents a column
                 with metadata such as name, type, nullability, primary key flag,
                 and server default configuration.

    Returns:
        A string containing the generated SQLAlchemy model class.
    """
    imports = set()
    type_imports = set()
    uses_text = False
    uses_func = False
    uses_postgresql = False
    lines = []

    INDENT = " " * 4

    for column in columns:
        name = column.get("name")
        col_type = column.get("type")
        primary_key = column.get("primary_key", False)
        nullable = column.get("nullable", True)
        server_default = column.get("server_default", False)
        default_function = column.get("default_function")

        if not name or not col_type:
            continue  # Skip malformed column definitions

        # Determine the SQLAlchemy type and import
        if col_type == "String":
            sa_type = "String"
            type_imports.add("String")
        elif col_type == "Integer":
            sa_type = "Integer"
            type_imports.add("Integer")
        elif col_type == "DateTime":
            sa_type = "DateTime"
            type_imports.add("DateTime")
        elif col_type == "Boolean":
            sa_type = "Boolean"
            type_imports.add("Boolean")
        elif col_type == "Float":
            sa_type = "Float"
            type_imports.add("Float")
        elif col_type == "UUID":
            sa_type = "UUID"
            type_imports.add("UUID")
            uses_postgresql = True
        else:
            continue  # Unsupported type

        # Build column definition
        col_parts = [f"{name} = Column({sa_type}"]

        if primary_key:
            col_parts.append("primary_key=True")
        if not nullable:
            col_parts.append("nullable=False")

        # Handle server_default
        if server_default:
            if isinstance(default_function, str):
                # Check for known SQL function defaults
                lowered = default_function.lower()
                if lowered in {"now", "current_timestamp", "uuid_generate_v4", "gen_random_uuid"}:
                    col_parts.append(f"server_default=func.{lowered}()")
                    uses_func = True
                else:
                    # Treat as raw SQL string literal
                    col_parts.append(f"server_default=text({default_function})")
                    uses_text = True
            elif isinstance(default_function, (int, float, bool)):
                val = str(default_function).upper() if isinstance(default_function, bool) else str(default_function)
                col_parts.append(f"server_default=text('{val}')")
                uses_text = True

        col_parts_str = ", ".join(col_parts) + ")"
        lines.append(INDENT + col_parts_str)

    # Prepare import lines
    imports.add("from sqlalchemy import Column")
    if type_imports:
        imports.add(f"from sqlalchemy import {', '.join(sorted(type_imports - {'UUID'}))}")
    if uses_text:
        imports.add("from sqlalchemy import text")
    if uses_func:
        imports.add("from sqlalchemy.sql import func")
    if uses_postgresql:
        imports.add("from sqlalchemy.dialects.postgresql import UUID")
    imports.add("from sqlalchemy.ext.declarative import declarative_base")

    # Assemble final output
    output = []
    output.extend(sorted(imports))
    output.append("")
    output.append("Base = declarative_base()")
    output.append("")
    output.append("class GeneratedModel(Base):")
    output.append(INDENT + "__tablename__ = 'generated_model'")
    if lines:
        output.extend(lines)
    else:
        output.append(INDENT + "pass")

    return "\n".join(output)

..............
----------------------------------------------------------------------
Ran 14 tests in 0.012s

OK


SystemExit: False

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# tests

import unittest

from textwrap import dedent

from main import generate_sqlalchemy_model  # Replace with the actual import if needed


class TestGenerateSQLAlchemyModel(unittest.TestCase):

    def test_empty_input(self):
        columns = []
        expected = dedent("""\
            from sqlalchemy import Column
            from sqlalchemy.ext.declarative import declarative_base

            Base = declarative_base()

            class GeneratedModel(Base):
                __tablename__ = 'generated_model'
                pass""")
        self.assertEqual(generate_sqlalchemy_model(columns).strip(), expected.strip())

    def test_basic_integer_column(self):
        columns = [{
            "name": "age",
            "type": "Integer",
            "primary_key": False,
            "nullable": True,
            "server_default": False,
            "default_function": None
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("age = Column(Integer)", output)

    def test_primary_key_uuid_with_func(self):
        columns = [{
            "name": "id",
            "type": "UUID",
            "primary_key": True,
            "nullable": False,
            "server_default": True,
            "default_function": "uuid_generate_v4"
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("UUID", output)
        self.assertIn("server_default=func.uuid_generate_v4()", output)

    def test_string_with_literal_default(self):
        columns = [{
            "name": "status",
            "type": "String",
            "primary_key": False,
            "nullable": False,
            "server_default": True,
            "default_function": "'active'"
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("server_default=text('active')", output)

    def test_datetime_with_now_function(self):
        columns = [{
            "name": "created_at",
            "type": "DateTime",
            "primary_key": False,
            "nullable": False,
            "server_default": True,
            "default_function": "now"
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("server_default=func.now()", output)

    def test_boolean_true_default(self):
        columns = [{
            "name": "is_active",
            "type": "Boolean",
            "primary_key": False,
            "nullable": True,
            "server_default": True,
            "default_function": True
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("server_default=text('TRUE')", output)

    def test_boolean_false_default(self):
        columns = [{
            "name": "is_active",
            "type": "Boolean",
            "primary_key": False,
            "nullable": True,
            "server_default": True,
            "default_function": False
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("server_default=text('FALSE')", output)

    def test_integer_literal_default(self):
        columns = [{
            "name": "level",
            "type": "Integer",
            "primary_key": False,
            "nullable": False,
            "server_default": True,
            "default_function": 5
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("server_default=text('5')", output)

    def test_float_literal_default(self):
        columns = [{
            "name": "rating",
            "type": "Float",
            "primary_key": False,
            "nullable": False,
            "server_default": True,
            "default_function": 4.5
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("server_default=text('4.5')", output)

    def test_unsupported_type_is_skipped(self):
        columns = [{
            "name": "metadata",
            "type": "JSONB",
            "primary_key": False,
            "nullable": True,
            "server_default": False,
            "default_function": None
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertNotIn("metadata", output)

    def test_missing_type_is_skipped(self):
        columns = [{
            "name": "broken",
            "primary_key": False,
            "nullable": True,
            "server_default": False,
            "default_function": None
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertNotIn("broken", output)

    def test_server_default_flag_true_but_missing_default_function(self):
        columns = [{
            "name": "email_verified",
            "type": "Boolean",
            "primary_key": False,
            "nullable": True,
            "server_default": True
            # default_function is missing
        }]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("email_verified", output)
        self.assertNotIn("server_default=", output)

    def test_all_supported_types(self):
        columns = [
            {"name": "id", "type": "UUID", "primary_key": True, "nullable": False, "server_default": True, "default_function": "uuid_generate_v4"},
            {"name": "name", "type": "String", "primary_key": False, "nullable": False, "server_default": True, "default_function": "'John'"},
            {"name": "age", "type": "Integer", "primary_key": False, "nullable": True, "server_default": True, "default_function": 30},
            {"name": "created_at", "type": "DateTime", "primary_key": False, "nullable": False, "server_default": True, "default_function": "CURRENT_TIMESTAMP"},
            {"name": "is_admin", "type": "Boolean", "primary_key": False, "nullable": False, "server_default": True, "default_function": False},
            {"name": "score", "type": "Float", "primary_key": False, "nullable": True, "server_default": True, "default_function": 98.6},
        ]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("id = Column(UUID", output)
        self.assertIn("name = Column(String", output)
        self.assertIn("age = Column(Integer", output)
        self.assertIn("created_at = Column(DateTime", output)
        self.assertIn("is_admin = Column(Boolean", output)
        self.assertIn("score = Column(Float", output)

    def test_import_lines_are_minimal_and_correct(self):
        columns = [
            {"name": "title", "type": "String", "primary_key": False, "nullable": False, "server_default": True, "default_function": "'draft'"},
        ]
        output = generate_sqlalchemy_model(columns)
        self.assertIn("from sqlalchemy import Column", output)
        self.assertIn("from sqlalchemy import String", output)
        self.assertIn("from sqlalchemy import text", output)
        self.assertNotIn("func.", output)



if __name__ == '__main__':
    unittest.main(argv = [''])



.......F......
FAIL: test_import_lines_are_minimal_and_correct (__main__.TestGenerateSQLAlchemyModel.test_import_lines_are_minimal_and_correct)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipython-input-5-3120688509.py", line 180, in test_import_lines_are_minimal_and_correct
    self.assertIn("from sqlalchemy import Column, String, text", output)
AssertionError: 'from sqlalchemy import Column, String, text' not found in "from sqlalchemy import Column\nfrom sqlalchemy import String\nfrom sqlalchemy import text\nfrom sqlalchemy.ext.declarative import declarative_base\n\nBase = declarative_base()\n\nclass GeneratedModel(Base):\n    __tablename__ = 'generated_model'\n    title = Column(String, nullable=False, server_default=text('draft'))"

----------------------------------------------------------------------
Ran 14 tests in 0.015s

FAILED (failures=1)


SystemExit: True

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/27c4bbf6-9a1e-4852-8251-05e000e6fca9?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67585%2Fview

#### Model code:

```python
from sqlalchemy import Column, Integer, String, DateTime, Boolean, Float, text
from sqlalchemy.dialects.postgresql import UUID
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql import func

def generate_sqlalchemy_model(columns):
    Base = declarative_base()
    
    # Dictionary to map string types to SQLAlchemy types
    type_map = {
        "Integer": Integer,
        "String": String,
        "DateTime": DateTime,
        "Boolean": Boolean,
        "UUID": UUID,
        "Float": Float
    }
    
    # Begin constructing the class as a series of strings
    imports = {
        "Column": "from sqlalchemy import Column",
        "declarative_base": "from sqlalchemy.ext.declarative import declarative_base"
    }
    lines = []
    lines.append("Base = declarative_base()")
    lines.append("")
    lines.append("class GeneratedModel(Base):")
    lines.append("    __tablename__ = 'generated_model'")
    lines.append("")

    for col in columns:
        name = col.get('name')
        col_type_str = col.get('type')
        primary_key = col.get('primary_key', False)
        nullable = col.get('nullable', True)
        server_default = col.get('server_default', False)
        default_function = col.get('default_function', None)
        
        if not name or not col_type_str:
            continue
        
        col_type = type_map.get(col_type_str)
        if not col_type:
            continue

        # Add necessary imports
        if col_type_str in ["UUID", "Integer", "String", "DateTime", "Boolean", "Float"]:
            imports[col_type_str] = f"from sqlalchemy import {col_type_str}"

        # Column definition string
        col_def = f"    {name} = Column({col_type.__name__}"

        # If type is String, ensure it's default length of 255 characters
        if col_type_str == "String":
            col_def += "(255)"
        
        options = []
        if primary_key:
            options.append("primary_key=True")
        if not nullable:
            options.append("nullable=False")

        if server_default:
            if isinstance(default_function, str):
                if default_function.lower() in {'now', 'current_timestamp', 'uuid_generate_v4', 'gen_random_uuid'}:
                    if "func" not in imports:
                        imports["func"] = "from sqlalchemy.sql import func"
                    options.append(f"server_default=func.{default_function}()")
                else:
                    if "text" not in imports:
                        imports["text"] = "from sqlalchemy import text"
                    options.append(f"server_default=text({default_function})")
            else:
                # Handle literal values like 0, True, False
                if "text" not in imports:
                    imports["text"] = "from sqlalchemy import text"
                if isinstance(default_function, bool):
                    sql_literal = 'TRUE' if default_function else 'FALSE'
                    options.append(f"server_default=text('{sql_literal}')")
                else:
                    options.append(f"server_default=text('{default_function}')")
# <Issue>: The model generated incomplete response resulting in unusable solution.
```